In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from torch.autograd import Variable

In [3]:
val = datasets.ImageFolder('val_128/',transform=data_transform_val)
valloader = torch.utils.data.DataLoader(val, batch_size=4,
                                         shuffle=True, num_workers=4)

test= test=datasets.ImageFolder('test_128/',transform=data_transform_val)
testloader = torch.utils.data.DataLoader(test,batch_size=4, shuffle=False, num_workers=4)

In [2]:
##,      transforms.Normalize(mean=[0.485, 0.456, 0.406],                          std=[0.229, 0.224, 0.225])
data_transform = transforms.Compose([transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor()
    ])
##,      transforms.Normalize(mean=[0.485, 0.456, 0.406],                          std=[0.229, 0.224, 0.225])
data_transform_val = transforms.Compose([transforms.Resize(224),
        transforms.ToTensor()
    ])

In [4]:
train = datasets.ImageFolder('train_128/',transform=data_transform)

In [5]:
dataset_loader = torch.utils.data.DataLoader(train,
                                             batch_size=4, shuffle=True,
                                             num_workers=4)

In [6]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo




class AlexNet(nn.Module):

    def __init__(self, num_classes=len(train.classes)):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


def alexnet(pretrained=False, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = AlexNet(**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['alexnet']))
    return model


In [ ]:
net = alexnet(pretrained=False).cuda()

In [7]:
alexNet = AlexNet()
alexNet.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096,

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexNet.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)

In [9]:
for epoch in range(60):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataset_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = Variable(inputs).cuda()
        labels = Variable(labels).cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        #print(inputs.shape)
        #print(len(inputs))
        # forward + backward + optimize
        outputs = alexNet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        # print statistics
        running_loss += loss.cuda()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 3.218
[2,  2000] loss: 3.183
[3,  2000] loss: 3.212
[4,  2000] loss: 3.120
[5,  2000] loss: 3.014
[6,  2000] loss: 2.760
[7,  2000] loss: 2.539
[8,  2000] loss: 2.160
[9,  2000] loss: 1.804
[10,  2000] loss: 1.565
[11,  2000] loss: 1.373
[12,  2000] loss: 1.226
[13,  2000] loss: 1.114
[14,  2000] loss: 1.015
[15,  2000] loss: 0.938
[16,  2000] loss: 0.879
[17,  2000] loss: 0.813
[18,  2000] loss: 0.777
[19,  2000] loss: 0.751
[20,  2000] loss: 0.698
[21,  2000] loss: 0.706
[22,  2000] loss: 0.658
[23,  2000] loss: 0.648
[24,  2000] loss: 0.605
[25,  2000] loss: 0.587
[26,  2000] loss: 0.563
[27,  2000] loss: 0.564
[28,  2000] loss: 0.565
[29,  2000] loss: 0.541
[30,  2000] loss: 0.528
[31,  2000] loss: 0.512
[32,  2000] loss: 0.505
[33,  2000] loss: 0.509
[34,  2000] loss: 0.494
[35,  2000] loss: 0.472
[36,  2000] loss: 0.458
[37,  2000] loss: 0.473
[38,  2000] loss: 0.471
[39,  2000] loss: 0.451
[40,  2000] loss: 0.438
[41,  2000] loss: 0.433
[42,  2000] loss: 0.433
[

In [10]:
true_val = []
pred_val = []
for data in valloader: 
    images_val, labels_val =data
    # print images
    
    #imshow(utils.make_grid(images_val))
    #print('GroundTruth: ', ' '.join('%5s' % val.classes[labels_val[j]] for j in range(4)))
    images_val = Variable(images_val).cuda()
    labels_val = Variable(labels_val).cuda()
    outputs_val = alexNet(images_val)
    _, predicted = torch.max(outputs_val, 1)
    true_val.append(labels_val)
    pred_val.append(predicted)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [1]:
len(true_val)

NameError: name 'true_val' is not defined

In [ ]:
len(pred_val)

In [ ]:
true_val[0]

In [12]:
correct =0
for (tr_batch, pr_batch) in zip(true_val,pred_val):
    v_pr = pr_batch.data[0]
    v_tr = tr_batch.data[0]
    if(v_pr==v_tr):
        correct+=1

In [13]:
correct/len(true_val)

0.8224

In [16]:
import pickle as pkl

In [35]:
with open('Alex60EpochWDecay.pt','rb') as f:
    filepklalex = pkl.load(f)

EOFError: Ran out of input

In [33]:
torch.load(pkl.load(filepklalex))

EOFError: Ran out of input

In [ ]:
np.save('alexnet',AlexNet)

In [16]:
import numpy as np

In [26]:

test_pred =[]
test_true = np.zeros(len(testloader))
for d in testloader:
    images_test, labels_test = data
    images_test = Variable(images_test).cuda()
    labels_test = Variable(labels_test).cuda()
    outputs_test = alexNet(images_test)
    _test, predicted_test = torch.max(outputs_test.data, 1)
    test_pred.append(predicted_test)
    
    

In [1]:
correct_test =0
for (pr_batch) in (true_val):
    v_pr = pr_batch.data[0]
    #v_tr = tr_batch.data[0]
    if(v_pr==0):
        correct_test+=1

NameError: name 'true_val' is not defined

In [23]:
len(test_pred)

625

In [29]:
for i in testloader:

Process Process-271:
Process Process-269:
Traceback (most recent call last):
Process Process-270:
Traceback (most recent call last):
Process Process-272:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 5

KeyboardInterrupt: 

  File "/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py", line 61, in __call__
    return F.to_tensor(pic)
  File "/usr/local/lib/python3.5/dist-packages/torchvision/transforms/functional.py", line 74, in to_tensor
    img = img.transpose(0, 1).transpose(0, 2).contiguous()
  File "/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py", line 147, in __call__
    return F.resize(img, self.size, self.interpolation)
KeyboardInterrupt
  File "/usr/local/lib/python3.5/dist-packages/torchvision/transforms/functional.py", line 197, in resize
    return img.resize((ow, oh), interpolation)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 355, in put
    self._writer.send_bytes(obj)
  File "/usr/local/lib/python3.5/dist-packages/PIL/Image.py", line 1749, in resize
    return self._new(self.im.resize(size, resample, box))
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offs

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte